In [1]:
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import pandas as pd

In [2]:
paths = ["data/CK+/","data/fer/train/"]
# data  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs = []
state = []

In [3]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state.extend([item for p in listdir(f"{path}{item}")])

In [4]:
len(state),len(imgs)

(29611, 29611)

In [5]:
imgs[:5],state[:5]

(['data/CK+/surprise/S052_001_00000015.png',
  'data/CK+/surprise/S050_002_00000018.png',
  'data/CK+/surprise/S086_001_00000019.png',
  'data/CK+/surprise/S037_001_00000020.png',
  'data/CK+/surprise/S034_001_00000029.png'],
 ['surprise', 'surprise', 'surprise', 'surprise', 'surprise'])

In [6]:
imgs_ = []
for p in imgs:
    temp = Image.open(p)
    save = temp.copy()
    imgs_.append(save)
    temp.close()


In [7]:
imgs_array = []
HEIGHT, WIDTH =48,48
for f in imgs_:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array.append(np.array(img))

In [8]:
#imgs_array = [el/255 for el in imgs_array]


In [9]:
imgs_array = np.array(imgs_array)

In [10]:
imgs_array.shape[1:]

(48, 48, 3)

In [11]:
categories = list(set(state))

In [12]:
categories

['fear', 'neutral', 'anger', 'surprise', 'disgust', 'sadness', 'happiness']

In [13]:
y=pd.DataFrame(state)
y_dummies = pd.get_dummies(y)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_array,y_dummies)

In [15]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD, Adamax
from tensorflow.keras.applications import ResNet50
import keras

In [16]:
opt = Adam(learning_rate=0.01)

In [17]:
model_resnet= ResNet50(weights=None,include_top=False, input_shape= (HEIGHT, WIDTH,3))

2021-11-12 16:55:59.741566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 16:55:59.744731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 16:55:59.744968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 16:55:59.745360: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [18]:
# model = Sequential()
# #model.add(Flatten())
# model.add(Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(imgs_array.shape[1:])))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Conv2D(filters=512, kernel_size=3, activation='relu', input_shape=(imgs_array.shape[1:])))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Flatten())
# model.add(Dense(64,activation = 'relu'))
# model.add(Dense(128,activation = 'relu'))
# model.add(Dense(256,activation = 'relu'))
# model.add(Dense(512,activation = 'relu'))
# model.add(Dropout(0.3))
# model.add(BatchNormalization())
# model.add(Dense(7, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])

In [19]:
output = Sequential()
output.add(Flatten())
output.add(Dense(64,activation = 'relu'))
output.add(Dense(128,activation = 'relu'))
output.add(Dense(256,activation = 'relu'))
output.add(Dense(512,activation = 'relu'))
output.add(Dropout(0.3))
output.add(BatchNormalization())
output.add(Dense(7, activation='softmax'))

In [20]:

model = Sequential([model_resnet,output])

model.compile(loss='categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])

In [21]:
from keras.models import Sequential, load_model, model_from_json
from keras import callbacks, optimizers
import tensorflow as tf
from datetime import date

fecha=str(date.today().year)+str(date.today().month)+str(date.today().day)    
symbol = 'complet_faces'
h5 = symbol + '_best_model' + fecha + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='loss',
                                       verbose=0,
                                       save_best_only=True,
                                       #save_weights_only=True,
                                       mode='auto',
                                       save_freq=1)
callback = [checkpoint]
json = symbol + '_best_model' + fecha + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)

modelo = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 5000,callbacks = callback,validation_split = 0.1)

/home/oobonioo/miniconda3/envs/core/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2021-11-12 16:54:55.920411: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 131.75MiB (rounded to 138150144)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2021-11-12 16:54:55.920458: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2021-11-12 16:54:55.920470: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 77, Chunks in use: 77. 19.2

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
paths = ["data/predict/","data/fer/test/"]
# data_pred  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs_pred = []
state_pred = []

In [ ]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs_pred.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state_pred.extend([item for p in listdir(f"{path}{item}")])



In [ ]:
imgs_pred

In [ ]:
imgs_pred_ = []
for p in imgs_pred:
    temp = Image.open(p)
    save = temp.copy()
    imgs_pred_.append(save)
    temp.close()

imgs_array_pred = []
for f in imgs_pred_:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array_pred.append(np.array(img))
imgs_array_pred = np.array(imgs_array_pred)

In [ ]:
prediction = model.predict(imgs_array_pred)

In [ ]:
len(prediction)

In [ ]:
prediction[:5]

In [ ]:
prediction.argmax(axis = -1)


In [ ]:
cats= y_dummies.columns
cats = [x.replace("0_","") for x in cats]

In [ ]:
cats

In [ ]:
states_model = []
for i in range(len(prediction)):
    states_model.append(cats[prediction[i].argmax()])
    print(state_pred[i],cats[prediction[i].argmax()],imgs_pred[i])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(state_pred,states_model)